<a href="https://colab.research.google.com/github/louis030195/colabs/blob/master/Pytorch_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [0]:
os.makedirs('images', exist_ok=True)

In [0]:
N_EPOCHS = 200
BATCH_SIZE=64
LEARNING_RATE=0.0002
B1 = 0.5
B2 = 0.999
N_CPU = 8
LATENT_DIM = 100
IMG_SIZE = 28
CHANNELS = 1
SAMPLE_INTERVAL = 400

In [0]:
img_shape = (CHANNELS, IMG_SIZE, IMG_SIZE)

cuda = True if torch.cuda.is_available() else False

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(LATENT_DIM, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Configure data loader
os.makedirs('../../data/mnist', exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data/mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=LEARNING_RATE, betas=(B1, B2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=LEARNING_RATE, betas=(B1, B2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

# ----------
#  Training
# ----------

for epoch in range(N_EPOCHS):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], LATENT_DIM))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]" % (epoch, N_EPOCHS, i, len(dataloader),
                                                            d_loss.item(), g_loss.item()))

        batches_done = epoch * len(dataloader) + i
        if batches_done % SAMPLE_INTERVAL == 0:
            save_image(gen_imgs.data[:25], 'images/%d.png' % batches_done, nrow=5, normalize=True)

Processing...
Done!
[Epoch 0/200] [Batch 0/938] [D loss: 0.699673] [G loss: 0.702894]
[Epoch 0/200] [Batch 1/938] [D loss: 0.612168] [G loss: 0.702224]
[Epoch 0/200] [Batch 2/938] [D loss: 0.546077] [G loss: 0.702052]
[Epoch 0/200] [Batch 3/938] [D loss: 0.491816] [G loss: 0.702060]
[Epoch 0/200] [Batch 4/938] [D loss: 0.446124] [G loss: 0.702048]
[Epoch 0/200] [Batch 5/938] [D loss: 0.411671] [G loss: 0.702212]
[Epoch 0/200] [Batch 6/938] [D loss: 0.391585] [G loss: 0.702012]
[Epoch 0/200] [Batch 7/938] [D loss: 0.374254] [G loss: 0.701680]
[Epoch 0/200] [Batch 8/938] [D loss: 0.363590] [G loss: 0.700999]
[Epoch 0/200] [Batch 9/938] [D loss: 0.360214] [G loss: 0.699818]
[Epoch 0/200] [Batch 10/938] [D loss: 0.355790] [G loss: 0.698801]
[Epoch 0/200] [Batch 11/938] [D loss: 0.354329] [G loss: 0.696999]
[Epoch 0/200] [Batch 12/938] [D loss: 0.354353] [G loss: 0.694231]
[Epoch 0/200] [Batch 13/938] [D loss: 0.354020] [G loss: 0.691956]
[Epoch 0/200] [Batch 14/938] [D loss: 0.355603] [G l